In [1]:
import FinanceDataReader as fdr
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
with open('./csv_origin/only_code.csv', 'r') as f:
    code_data = f.readlines()
    tmp = []
    for line in code_data:
        tmp.extend(line.split(','))  # 각 라인을 쉼표로 분할하여 tmp 리스트에 추가합니다.
    code_data = tmp

for code in code_data:
    url = 'http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode='+code[:-1]+'&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
    response = requests.get(url)
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')

    data = []
    table = soup.find_all(class_ = 'us_table_ty1 h_fix zigbg_no')

    table = table[6]

    header = table.find_all('thead')[0].find_all('th')
    header_text = [th.find('div').get_text() for th in header]

    for row in table.find('tbody').find_all('tr'):
        tmp = [cell.text.strip() for cell in row.find_all('td')]
        data.append(tmp)

    columns = [th.text.strip() for th in table.find('tbody').find_all('th')]

    df = pd.DataFrame(data, columns)
    df=df.transpose()
    
    df.to_csv('./data_csv/'+code[:-1]+'.csv', index=False)